<a href="https://colab.research.google.com/github/Adithyasnr/Gas-Guardian-IOT/blob/main/ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to the folder inside Google Drive
folder_path = "/content/drive/My Drive/ECG ID"

# Check if the folder exists
if os.path.exists(folder_path):
    files = os.listdir(folder_path)  # List all files
    print(f"✅ Found {len(files)} files in '{folder_path}'")

    for file in files[:10]:  # Print only first 10 files as a preview
        print(file)
else:
    print("❌ Folder not found! Check the path.")


Mounted at /content/drive
✅ Found 91 files in '/content/drive/My Drive/ECG ID'
Person_85
Person_84
Person_88
Person_81
Person_83
Person_82
Person_86
Person_90
Person_87
Person_89


In [ ]:
pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
import os
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, BatchNormalization, Dropout

In [ ]:
!pip install PyWavelets
import pywt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.5 MB/s eta 0:00:00


In [ ]:
def load_ecg_data(record_path):
    record = wfdb.rdrecord(record_path)
    annotation = wfdb.rdann(record_path, 'atr')
    ecg_signal = record.p_signal[:, 0]
    r_peaks = annotation.sample

    # Apply baseline drift removal and bandpass filter
    ecg_signal = remove_baseline_drift(ecg_signal)
    ecg_signal = bandpass_filter(ecg_signal, low_freq=0.5, high_freq=40, fs=500)

    return ecg_signal, r_peaks

In [ ]:
def remove_baseline_drift(signal, wavelet="db6", level=9):
    coeff = pywt.wavedec(signal, wavelet, level=level)
    coeff[0] = np.zeros_like(coeff[0])  # Remove baseline drift
    reconstructed_signal = pywt.waverec(coeff, wavelet)
    return reconstructed_signal

In [ ]:
def bandpass_filter(signal, low_freq, high_freq, fs, order=5):
    nyquist = 0.5 * fs
    low = low_freq / nyquist
    high = high_freq / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

In [ ]:
def normalize_and_segment(signal, r_peaks, window_size=180):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    signal_normalized = scaler.fit_transform(signal.reshape(-1, 1)).flatten()

    segments = []
    half_window = window_size // 2
    for r_peak in r_peaks:
        start = max(r_peak - half_window, 0)
        end = min(r_peak + half_window, len(signal_normalized))
        if end - start == window_size:
            segments.append(signal_normalized[start:end])
    return np.array(segments)

In [ ]:
def load_and_process_all_records(base_directory, visualize=False):
    all_segments = []
    all_labels = []
    person_dirs = [d for d in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, d))]
    lis =  ['Person_01', 'Person_02','Person_09', 'Person_10' ]
    print(person_dirs)
    for person_id in lis:
        person_path = os.path.join(base_directory, person_id)
        for record_file in os.listdir(person_path):
            if record_file.endswith('.dat'):
                record_base = record_file[:-4]
                record_path = os.path.join(person_path, record_base)
                ecg_signal, r_peaks = load_ecg_data(record_path)
                segments = normalize_and_segment(ecg_signal, r_peaks)
                all_segments.extend(segments)
                all_labels.extend([person_id] * len(segments))
                if visualize and person_id == 'Person_01':  # Example visualization for one person
                    visualize_ecg_signals(ecg_signal, segments, r_peaks, record_base)

    return np.array(all_segments), np.array(all_labels)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
all_segments, all_labels = load_and_process_all_records(folder_path)

# Encoding labels
label_encoder = LabelEncoder()
all_labels_encoded = label_encoder.fit_transform(all_labels)

# Prepare data for the LSTM model
X = np.array(all_segments).reshape((len(all_segments), -1, 1))
y = np.array(all_labels_encoded)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the BiLSTM model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),  # Adding dropout for regularization
    BatchNormalization(),  # Normalize the activations from the LSTM layer

    Bidirectional(LSTM(512, return_sequences=True)),
    # Additional LSTM layer with sequence return
    Dropout(0.2),  # More dropout for regularization
    BatchNormalization(),  # More normalization
     Bidirectional(LSTM(256, return_sequences=True)),
    # Additional LSTM layer with sequence return
    Dropout(0.2),  # More dropout for regularization
    BatchNormalization(),
     Bidirectional(LSTM(128, return_sequences=True)),
    # Additional LSTM layer with sequence return
    Dropout(0.2),  # More dropout for regularization
    BatchNormalization(),
    Bidirectional(LSTM(64)),  # Final LSTM layer without sequence return
#     Dropout(0.2),  # Dropout after final LSTM layer
    Dense(128, activation='relu'),  # An additional Dense layer for more complex transformations
#     Dropout(0.2),  # Dropout after Dense layer
    Dense(len(np.unique(y_train)), activation='softmax')  # Output layer
])

optimizer = RMSprop(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')

['Person_85', 'Person_84', 'Person_88', 'Person_81', 'Person_83', 'Person_82', 'Person_86', 'Person_90', 'Person_87', 'Person_89', 'Person_75', 'Person_79', 'Person_71', 'Person_72', 'Person_73', 'Person_77', 'Person_78', 'Person_76', 'Person_74', 'Person_80', 'Person_63', 'Person_66', 'Person_69', 'Person_70', 'Person_64', 'Person_61', 'Person_62', 'Person_67', 'Person_68', 'Person_65', 'Person_51', 'Person_54', 'Person_53', 'Person_56', 'Person_52', 'Person_58', 'Person_60', 'Person_57', 'Person_55', 'Person_59', 'Person_43', 'Person_49', 'Person_50', 'Person_47', 'Person_42', 'Person_41', 'Person_48', 'Person_45', 'Person_46', 'Person_44', 'Person_37', 'Person_38', 'Person_34', 'Person_36', 'Person_33', 'Person_40', 'Person_31', 'Person_32', 'Person_39', 'Person_35', 'Person_27', 'Person_21', 'Person_30', 'Person_25', 'Person_24', 'Person_28', 'Person_26', 'Person_29', 'Person_23', 'Person_22', 'Person_13', 'Person_20', 'Person_15', 'Person_14', 'Person_12', 'Person_11', 'Person_17'

KeyboardInterrupt: 

In [ ]:
model.save('/content/drive/My Drive/ECG ID/ecg_identification_model_bilstm.h5')
model.save('/content/ecg_identification_model_bilstm.h5')

In [ ]:
from sklearn.metrics import classification_report

#  `y_test` are the true labels and `y_pred` are the predicted labels from model
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate classification report
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print(report)


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step
              precision    recall  f1-score   support

   Person_01       0.92      1.00      0.96        79
   Person_02       0.87      0.89      0.88        84
   Person_09       0.92      0.85      0.88        26
   Person_10       0.90      0.67      0.77        27

    accuracy                           0.90       216
   macro avg       0.90      0.85      0.87       216
weighted avg       0.90      0.90      0.90       216



In [ ]:
import numpy as np
import random
random_index = random.randint(0, len(X_test) - 1)
random_sample = X_test[random_index]
true_class = y_test[random_index]

predicted_probs = model.predict(np.expand_dims(random_sample, axis=0))
predicted_class = np.argmax(predicted_probs)

predicted_person = label_encoder.inverse_transform([predicted_class])[0]
true_person = label_encoder.inverse_transform([true_class])[0]

print(f"Random Sample Index: {random_index}")
if predicted_person in ["Person_01", "Person_02", "Person_09", "Person_10"]:
    print("Authentication Success")
else:
    print("Authentication failed")
print("WELCOME ",predicted_person)
print("\n")
print(f"Predicted Class: {predicted_class} ({predicted_person})")
print(f"True Class: {true_class} ({true_person})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Random Sample Index: 39
Authentication Success
WELCOME  Person_10


Predicted Class: 3 (Person_10)
True Class: 3 (Person_10)
